# Function that generates the timestamp range of when something important happens

#### i.e. when the shrimp are "close" to each other
#### The function takes in a dataframe that is outputted throug tracktor, and identifies when the two objectd are close to each other within a certain threshhold of distance

In [1]:
# First, load necessary packages 

import numpy as np
import pandas as pd
import tracktor as tr
import cv2
import sys
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

In [2]:
# Then, read in the "timestamped" CSV file from whatever directory 

df = pd.read_csv('/Users/lukefields/Downloads/BlackWhiteResult2full_tracked2.csv')
df = df.drop(["Unnamed: 0"], axis = 1)

#### We are going to try out the ranges now, by having a sort of recursive function that looks to see if the previous distance has exited the "important" range we have established

In [3]:
df["Previous"] = (df["Distance"].shift(1)).fillna(df['Distance'])
df

#### Now, we need to establish an "Enter" and "Exit" type of range generation

In [4]:
ashley_df["Enter"] = ashley_df.eval("Distance <= 130 and not (Previous <= 130)").cumsum()
ashley_df["Exit"] = ashley_df.eval("Previous <= 130 and not (Distance <= 130)")\
.shift(-1).fillna(True).cumsum()

#### Generate when it starts and ends 

In [5]:
temp = ashley_df.merge(ashley_df.drop_duplicates("Exit", keep = "first")[["Frame", "Exit"]],
                      left_on = "Enter", right_on = "Exit", how = "left", suffixes = ["", "_end"])

#### Dropping the duplicates identifies the unique different interval series, and BOOM

In [6]:
temp = temp.drop_duplicates(subset = "Enter", keep = "first")[["Frame", "Frame_end"]]
temp

,Frame,Frame_end
0,3.0,3.0
51,54.0,52.0
53,56.0,54.0
65,68.0,60.0
69,72.0,68.0
87,90.0,73.0
102,105.0,90.0
105,108.0,106.0


In [7]:
ashley_df = pd.read_csv('/Users/lukefields/Desktop/Shrimp-Capstone/shrimp_data_out/ashley_test_timestamped.csv')
ashley_df = ashley_df.drop(["Unnamed: 0"], axis = 1)
let_say = 125

ashley_df["Previous"] = ashley_df["Distance"].shift(1)
ashley_df

enter_eval = "Distance <= " + str(let_say) + " and not (Previous <= " + str(let_say) + ")"
exit_eval = "Previous <= " + str(let_say) + " and not (Distance <= " + str(let_say) + ")"
ashley_df["Enter"] = ashley_df.eval(enter_eval).cumsum()
ashley_df["Exit"] = ashley_df.eval(exit_eval).shift(-1).fillna(True).cumsum()

temp = ashley_df.merge(ashley_df.drop_duplicates("Exit", keep = "first")[["Frame", "Exit"]],
                      left_on = "Enter", right_on = "Exit", how = "left", suffixes = ["", "_end"])

temp = temp.drop_duplicates(subset = "Enter", keep = "first")[["Frame", "Frame_end"]]
temp

,Frame,Frame_end
0,3.0,3.0
1,4.0,5.0
6,9.0,9.0
12,15.0,16.0
31,34.0,36.0


# Ok, so how do we automate this process into one function?

In [8]:
ashley_df["Previous"] = (ashley_df["Distance"].shift(1)).fillna(ashley_df['Distance'])
ashley_df

,Frame,Timestamp,Distance,Previous,Enter,Exit
0,3.0,0.050000,126.321020,126.321020,0,0
1,4.0,0.066667,122.200655,126.321020,1,0
2,5.0,0.083333,124.145076,122.200655,1,1
3,6.0,0.100000,127.192767,124.145076,1,1
4,7.0,0.116667,127.251719,127.192767,1,1
...,...,...,...,...,...,...
164,167.0,2.783333,135.369864,133.304163,4,4
165,168.0,2.800000,138.293167,135.369864,4,4
166,169.0,2.816667,144.886162,138.293167,4,4
167,170.0,2.833333,146.874096,144.886162,4,4


In [9]:
df = pd.DataFrame(columns = ['Name', 'Articles', 'Improved'],
        index = ['a', 'b', 'c'])
  
print("Empty DataFrame With NaN values : \n\n", df)
  
# adding rows to an empty
# dataframe at existing index
df.loc['a'] = ['Ankita', 50, 100]
df.loc['b'] = ['Ankit', 60, 120]
df.loc['c'] = ['Harsh', 30, 60]
  

Empty DataFrame With NaN values : 

   Name Articles Improved
a  NaN      NaN      NaN
b  NaN      NaN      NaN
c  NaN      NaN      NaN


In [10]:
def important_frame(df, dist_cutoff):
    if ((df.loc[df["Distance"] < dist_cutoff]).empty == True):
        df_temp = pd.DataFrame(columns = ["Pair", "Frame", "Frame_end"],
                              index = [0])
        df_pair = df["Pair"][0]
        df_temp.loc[0] = [df_pair, 0, 0]
        return df_temp
    else:
        df["Previous"] = (df["Distance"].shift(1)).fillna(ashley_df['Distance'])

        enter_eval = "Distance <= " + str(dist_cutoff) + " and not (Previous <= " + str(dist_cutoff) + ")"
        exit_eval = "Previous <= " + str(dist_cutoff) + " and not (Distance <= " + str(dist_cutoff) + ")"
        df["Enter"] = df.eval(enter_eval).cumsum()
        df["Exit"] = df.eval(exit_eval).shift(-1).fillna(True).cumsum()

        df_temp = df.merge(df.drop_duplicates("Exit", keep = "first")[["Frame", "Exit"]],
                              left_on = "Enter", right_on = "Exit", how = "left", suffixes = ["", "_end"])

        df_temp = df_temp.drop_duplicates(subset = "Enter", keep = "first")[["Frame", "Frame_end"]]
        return df_temp

In [11]:
important_frame(ashley_df, 1)

KeyError: 'Pair'

In [12]:
def important_frames(df, dist_cutoff):
    df["Previous"] = (df["Distance"].shift(1)).fillna(ashley_df['Distance'])

    enter_eval = "Distance <= " + str(dist_cutoff) + " and not (Previous <= " + str(dist_cutoff) + ")"
    exit_eval = "Previous <= " + str(dist_cutoff) + " and not (Distance <= " + str(dist_cutoff) + ")"
    df["Enter"] = df.eval(enter_eval).cumsum()
    df["Exit"] = df.eval(exit_eval).shift(-1).fillna(True).cumsum()

    df_temp = df.merge(df.drop_duplicates("Exit", keep = "first")[["Frame", "Exit"]],
                          left_on = "Enter", right_on = "Exit", how = "left", suffixes = ["", "_end"])

    df_temp = df_temp.drop_duplicates(subset = "Enter", keep = "first")[["Frame", "Frame_end"]]
    return df_temp

In [13]:
important_frames(ashley_df, 124)

,Frame,Frame_end
0,3.0,3.0
1,4.0,4.0
6,9.0,9.0
31,34.0,35.0


In [14]:
import glob
import os

# The function below takes in every individual distance dataframe generated 
# And then concatentes them into one


# Only parameter is the filepath where each individual pair csv file is stores (Shrimp A and B)
def combine_dfs(filepath):
    all_files = glob.glob(os.path.join(filepath, "*.csv"))

    df_storage = []
    for filepath in all_files:
        individual_df = pd.read_csv(filepath, index_col = None, header = 0)
        df_storage.append(individual_df)

    combined_df = pd.concat(df_storage, axis = 0, ignore_index = True)
    combined_df = combined_df.drop(["Unnamed: 0"], axis = 1).sort_values("Pair", ascending = True)

    return combined_df

test_path = "/Users/lukefields/Desktop/Shrimp-Capstone/test_data_out/"
zebra_comb = combine_dfs(test_path)

In [23]:
zebra_comb

,Pair,Frame,Timestamp,Distance
149979,"('A', 'B')",15000.0,0:04:10: 0,50.675908
139987,"('A', 'B')",5008.0,0:01:23:46,96.587055
139986,"('A', 'B')",5007.0,0:01:23:45,96.969558
139985,"('A', 'B')",5006.0,0:01:23:43,98.314128
139984,"('A', 'B')",5005.0,0:01:23:41,100.498698
...,...,...,...,...
84983,"('D', 'E')",9996.0,0:02:46:59,226.578061
84982,"('D', 'E')",9995.0,0:02:46:58,225.084797
84981,"('D', 'E')",9994.0,0:02:46:56,222.018402
84993,"('D', 'E')",10006.0,0:02:46:76,249.971253


In [24]:
shrimp_df_0308 = pd.read_csv('/Users/lukefields/Desktop/Shrimp-Capstone/test_0308/AB_0308_distances.csv')
shrimp_df_0308 = shrimp_df_0308.drop(["Unnamed: 0"], axis = 1)

In [25]:
zebra_comb["Pair"][2]

"('C', 'E')"

In [51]:
def important_timestamps(df, dist_cutoff):
    if ((df.loc[df["Distance"] < dist_cutoff]).empty == True):
        df_temp = pd.DataFrame(columns = ["Pair", "Timestamp", "Timestamp_end"],
                              index = [0])
        df_pair = df["Pair"][0]
        df_temp.loc[0] = [df_pair, 0, 0]
        return df_temp
    else:
        df["Previous"] = (df["Distance"].shift(1))

        enter_eval = "Distance <= " + str(dist_cutoff) + " and not (Previous <= " + str(dist_cutoff) + ")"
        exit_eval = "Previous <= " + str(dist_cutoff) + " and not (Distance <= " + str(dist_cutoff) + ")"
        df["Enter"] = df.eval(enter_eval).cumsum()
        df["Exit"] = df.eval(exit_eval).shift(-1).fillna(True).cumsum()

        df_temp = df.merge(df.drop_duplicates("Exit", keep = "first")[["Timestamp", "Exit"]],
                              left_on = "Enter", right_on = "Exit", how = "left", suffixes = ["", "_end"])

        df_temp = df_temp.drop_duplicates(subset = "Enter", keep = "first")[["Pair", "Timestamp", "Timestamp_end"]]
        if ((df.loc[df["Distance"][0]]) < dist_cutoff) == True:
            return df_temp
        else:
            df_temp = df_temp.loc[1:]
            return df_temp

In [52]:
shrimp_df_AC = pd.read_csv('/Users/lukefields/Desktop/Shrimp-Capstone/test_data_out/AC_zebra_distances.csv')
shrimp_df_AC = shrimp_df_AC.drop(["Unnamed: 0"], axis = 1)

In [59]:
def timestamp_dfs(filepath):
    all_files = glob.glob(os.path.join(filepath, "*.csv"))

    df_storage = []
    for filepath in all_files:
        individual_df = pd.read_csv(filepath, index_col = None, header = 0)
        timestamped_df = important_timestamps(individual_df, 10)
        df_storage.append(timestamped_df)

    combined_df = pd.concat(df_storage, axis = 0, ignore_index = True)
    combined_df = combined_df.sort_values("Pair", ascending = True)

    return combined_df

test_path = "/Users/lukefields/Desktop/Shrimp-Capstone/test_data_out/"
zebra_ts = timestamp_dfs(test_path)
zebra_ts

,Pair,Timestamp,Timestamp_end
10,"('A', 'B')",0,0
9,"('A', 'C')",0:03:50:61,0:03:50:61
7,"('A', 'D')",0,0
8,"('A', 'E')",0,0
2,"('B', 'C')",0:01:43: 8,0:01:43: 8
3,"('B', 'C')",0:02:00:25,0:02:00:25
5,"('B', 'D')",0,0
4,"('B', 'E')",0,0
1,"('C', 'D')",0,0
0,"('C', 'E')",0,0


In [57]:
shrimp_df_AC["Distance"][0] < 10

False

In [58]:
def important_timestamps(df, dist_cutoff):
    if ((df.loc[df["Distance"] < dist_cutoff]).empty == True):
        df_temp = pd.DataFrame(columns = ["Pair", "Timestamp", "Timestamp_end"],
                              index = [0])
        df_pair = df["Pair"][0]
        df_temp.loc[0] = [df_pair, 0, 0]
        return df_temp
    else:
        df["Previous"] = (df["Distance"].shift(1))

        enter_eval = "Distance <= " + str(dist_cutoff) + " and not (Previous <= " + str(dist_cutoff) + ")"
        exit_eval = "Previous <= " + str(dist_cutoff) + " and not (Distance <= " + str(dist_cutoff) + ")"
        df["Enter"] = df.eval(enter_eval).cumsum()
        df["Exit"] = df.eval(exit_eval).shift(-1).fillna(True).cumsum()

        df_temp = df.merge(df.drop_duplicates("Exit", keep = "first")[["Timestamp", "Exit"]],
                              left_on = "Enter", right_on = "Exit", how = "left", suffixes = ["", "_end"])

        df_temp = df_temp.drop_duplicates(subset = "Enter", keep = "first")[["Pair", "Timestamp", "Timestamp_end"]]
        if df["Distance"][0] < dist_cutoff == True:
            return df_temp
        else:
            df_temp = df_temp.loc[1:]
            return df_temp


important_timestamps(shrimp_df_AC, 10)

,Pair,Timestamp,Timestamp_end
13834,"('A', 'C')",0:03:50:61,0:03:50:61
